In [1]:
import joblib

In [3]:
data_path = "/home/jianrenw/MAP/data/isaac_adam_standard_add_sign.pt"
adam_poses = joblib.load(data_path)

In [67]:
key = "KIT_3_walking_medium02_poses"
adam_pose = adam_poses[key]

In [4]:
from isaacgym import gymapi, gymtorch, gymutil

Importing module 'gym_38' (/home/jianrenw/isaacgym/python/isaacgym/_bindings/linux-x86_64/gym_38.so)
Setting GYM_USD_PLUG_INFO_PATH to /home/jianrenw/isaacgym/python/isaacgym/_bindings/linux-x86_64/usd/plugInfo.json


/home/jianrenw/anaconda3/envs/MAP/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PyTorch version 1.13.1
Device count 4
/home/jianrenw/isaacgym/python/isaacgym/_bindings/src/gymtorch


Using /home/jianrenw/.cache/torch_extensions/py38_cu116 as PyTorch extensions root...
Emitting ninja build file /home/jianrenw/.cache/torch_extensions/py38_cu116/gymtorch/build.ninja...
Building extension module gymtorch...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.


Loading extension module gymtorch...


In [6]:
gym = gymapi.acquire_gym()

sim = gym.create_sim()

# add cartpole urdf asset
asset_root = "robots/adam_standard"
asset_file = "urdf/adam_standard_foot_contact.urdf"

asset_options = gymapi.AssetOptions()
asset_options.collapse_fixed_joints = False

robot_asset = gym.load_asset(sim, asset_root, asset_file)
rigid_body_names = gym.get_asset_rigid_body_names(robot_asset)
rigid_body_dict = gym.get_asset_rigid_body_dict(robot_asset)
dof_names = gym.get_asset_dof_names(robot_asset)
left_foot_indices = [i for i, name in enumerate(rigid_body_names) if 'leftFoot' in name]
right_foot_indices = [i for i, name in enumerate(rigid_body_names) if 'rightFoot' in name]

Physics Engine: FleX
Physics Device: cuda:0
GPU Pipeline: disabled


In [7]:
left_foot_indices

[7, 8, 9, 10]

In [8]:
right_foot_indices

[17, 18, 19, 20]

In [10]:
import torch

In [9]:
spacing = 2.0
lower = gymapi.Vec3(-spacing, -spacing, -spacing)
upper = gymapi.Vec3(spacing, spacing, spacing)
env = gym.create_env(sim, lower, upper, 1)

pose = gymapi.Transform()
pose.p = gymapi.Vec3(0.0, 0.0, 2.0)

actor_handle = gym.create_actor(env, robot_asset, pose, "adam_actor", 0, 1)

In [68]:
root_pos = adam_pose["root_pos"]
root_rot = adam_pose["root_rot"]
root_vel = adam_pose["root_vel"]
root_angular_vel = adam_pose["root_angular_vel"]
dof_pos = adam_pose["dof_pos"]
dof_vel = adam_pose["dof_vel"]

frame_num = len(root_pos)

root_states = torch.cat([torch.from_numpy(root_pos), torch.from_numpy(root_rot), torch.from_numpy(root_vel), torch.from_numpy(root_angular_vel)], dim=1).type(torch.float32)
dof_states = torch.stack([torch.from_numpy(dof_pos), torch.from_numpy(dof_vel)],axis=2).type(torch.float32)
rigid_body_states = []

# Simulate
new_root_poses = []

for i in range(frame_num):

    gym.set_actor_root_state_tensor(sim, gymtorch.unwrap_tensor(root_states[i]))
    gym.set_dof_state_tensor(sim, gymtorch.unwrap_tensor(dof_states[i]))
    gym.refresh_rigid_body_state_tensor(sim)
    rigid_body_state = gym.acquire_rigid_body_state_tensor(sim)
    rigid_body_state = gymtorch.wrap_tensor(rigid_body_state)
    left_foot_poses = rigid_body_state[left_foot_indices, 0:3]
    right_foot_poses = rigid_body_state[right_foot_indices, 0:3]
    new_root_pose = root_states[i, 0:3]
    new_root_pose[2] = root_states[i, 2] - torch.min(torch.cat([left_foot_poses, right_foot_poses])[:,2])
    new_root_poses.append(new_root_pose)
    print(torch.min(torch.cat([left_foot_poses, right_foot_poses])[:,2]))
new_root_poses = torch.stack(new_root_poses, dim=0)

tensor(-0.0124)
tensor(-0.0124)
tensor(-0.0124)
tensor(-0.0123)
tensor(-0.0123)
tensor(-0.0122)
tensor(-0.0121)
tensor(-0.0120)
tensor(-0.0120)
tensor(-0.0120)
tensor(-0.0120)
tensor(-0.0119)
tensor(-0.0118)
tensor(-0.0117)
tensor(-0.0117)
tensor(-0.0117)
tensor(-0.0117)
tensor(-0.0118)
tensor(-0.0118)
tensor(-0.0119)
tensor(-0.0119)
tensor(-0.0118)
tensor(-0.0118)
tensor(-0.0117)
tensor(-0.0116)
tensor(-0.0115)
tensor(-0.0114)
tensor(-0.0114)
tensor(-0.0113)
tensor(-0.0113)
tensor(-0.0113)
tensor(-0.0112)
tensor(-0.0112)
tensor(-0.0112)
tensor(-0.0112)
tensor(-0.0113)
tensor(-0.0113)
tensor(-0.0113)
tensor(-0.0113)
tensor(-0.0113)
tensor(-0.0113)
tensor(-0.0113)
tensor(-0.0113)
tensor(-0.0113)
tensor(-0.0113)
tensor(-0.0113)
tensor(-0.0113)
tensor(-0.0113)
tensor(-0.0113)
tensor(-0.0113)
tensor(-0.0113)
tensor(-0.0113)
tensor(-0.0113)
tensor(-0.0114)
tensor(-0.0114)
tensor(-0.0114)
tensor(-0.0115)
tensor(-0.0115)
tensor(-0.0115)
tensor(-0.0116)
tensor(-0.0116)
tensor(-0.0115)
tensor(-

In [61]:
adam_pose["new_root_poses"] = new_root_poses.numpy()

In [62]:
adam_pose.keys()

dict_keys(['body_pos', 'root_pos', 'dof_pos', 'body_rot', 'root_rot', 'body_vel', 'root_vel', 'body_angular_vel', 'root_angular_vel', 'dof_vel', 'dt', 'dof_vel_sign', 'new_root_poses'])

In [63]:
joblib.dump(adam_pose, 'test.pt')

['test.pt']

In [64]:
new_root_poses.shape

torch.Size([329, 3])

In [66]:
root_pos = adam_pose["new_root_poses"]
root_rot = adam_pose["root_rot"]
root_vel = adam_pose["root_vel"]
root_angular_vel = adam_pose["root_angular_vel"]
dof_pos = adam_pose["dof_pos"]
dof_vel = adam_pose["dof_vel"]

frame_num = len(root_pos)

root_states = torch.cat([torch.from_numpy(root_pos), torch.from_numpy(root_rot), torch.from_numpy(root_vel), torch.from_numpy(root_angular_vel)], dim=1).type(torch.float32)
dof_states = torch.stack([torch.from_numpy(dof_pos), torch.from_numpy(dof_vel)],axis=2).type(torch.float32)
rigid_body_states = []

# Simulate
new_root_poses = []

for i in range(frame_num):

    gym.set_actor_root_state_tensor(sim, gymtorch.unwrap_tensor(root_states[i]))
    gym.set_dof_state_tensor(sim, gymtorch.unwrap_tensor(dof_states[i]))
    gym.refresh_rigid_body_state_tensor(sim)
    rigid_body_state = gym.acquire_rigid_body_state_tensor(sim)
    rigid_body_state = gymtorch.wrap_tensor(rigid_body_state)
    left_foot_poses = rigid_body_state[left_foot_indices, 0:3]
    right_foot_poses = rigid_body_state[right_foot_indices, 0:3]
    print(torch.min(torch.cat([left_foot_poses, right_foot_poses])[:,2]))

tensor(-6.7055e-08)
tensor(7.4506e-09)
tensor(7.4506e-09)
tensor(1.4901e-08)
tensor(-7.4506e-09)
tensor(-2.2352e-08)
tensor(-2.9802e-08)
tensor(-8.1956e-08)
tensor(-7.4506e-09)
tensor(2.2352e-08)
tensor(-7.4506e-09)
tensor(1.4901e-08)
tensor(-2.2352e-08)
tensor(-1.4901e-08)
tensor(2.2352e-08)
tensor(-1.4901e-08)
tensor(-7.4506e-09)
tensor(-5.2154e-08)
tensor(-3.7253e-08)
tensor(5.2154e-08)
tensor(0.)
tensor(0.)
tensor(-7.4506e-09)
tensor(0.)
tensor(-2.9802e-08)
tensor(-1.4901e-08)
tensor(7.4506e-09)
tensor(4.4703e-08)
tensor(0.)
tensor(7.4506e-09)
tensor(-7.4506e-09)
tensor(2.2352e-08)
tensor(-2.2352e-08)
tensor(2.2352e-08)
tensor(8.1956e-08)
tensor(2.2352e-08)
tensor(7.4506e-09)
tensor(-3.7253e-08)
tensor(-1.4901e-08)
tensor(-1.4901e-08)
tensor(-1.4901e-08)
tensor(-7.4506e-09)
tensor(1.4901e-08)
tensor(1.4901e-08)
tensor(2.9802e-08)
tensor(-3.7253e-08)
tensor(1.4901e-08)
tensor(-8.1956e-08)
tensor(2.2352e-08)
tensor(7.4506e-09)
tensor(-2.2352e-08)
tensor(-7.4506e-09)
tensor(-7.4506e-0